In [101]:
import os
import librosa
import numpy as np
import sklearn
import pandas as pd
from models.linear_regression import LinearRegression

In [102]:
model = LinearRegression()

In [103]:
DATASET_PATH = "LivingRoom_preprocessed_hack/Human1"

centroid = np.load(os.path.join(DATASET_PATH, "centroid.npy"))
print("Shape of Centroid:")
print(centroid.shape)

#Loading Room Impulse Response (1000 human locations x 10 microphones x M time samples)
RIRs = np.load(os.path.join(DATASET_PATH, "deconvoled_trim.npy"), mmap_mode='r')
print("Shape of RIRs:")
print(RIRs.shape)

Shape of Centroid:
(1000, 2)
Shape of RIRs:
(1000, 4, 667200)


In [104]:
rms_values = np.sqrt(np.mean(RIRs[0,0]**2, axis=-1))
rms_values

0.0005250508

In [105]:
targets = list(map(tuple,centroid))


In [106]:
def rms(x):
    return np.sqrt(np.mean(x**2, axis=-1))

In [107]:
c1 = []
c2 = []
c3 = []
c4 = []
#iterate through all data
for i in range(RIRs.shape[0]):
    c1.append(rms(RIRs[i,0]))
    c2.append(rms(RIRs[i,1]))
    c3.append(rms(RIRs[i,2]))
    c4.append(rms(RIRs[i,3]))

Dict = {"c1":c1,"c2":c2,"c3":c3, "c4":c4, "target":targets}
dataset = pd.DataFrame(data=Dict)
dataset

,c1,c2,c3,c4,target
0,0.000525,0.000502,0.000708,0.000430,"(-3231.329346696207, -1127.877714566052)"
1,0.000521,0.000500,0.000707,0.000429,"(-3198.541078748347, -744.5100655962758)"
2,0.000522,0.000499,0.000705,0.000429,"(-3192.977627395698, -248.26678826687157)"
3,0.000521,0.000492,0.000706,0.000426,"(-3018.418252755987, 334.9385460800696)"
4,0.000520,0.000491,0.000699,0.000429,"(-3015.7795323261244, 826.4402007987115)"
...,...,...,...,...,...
995,0.000516,0.000488,0.000688,0.000419,"(-2232.6553022604044, -3227.7906840139867)"
996,0.000514,0.000489,0.000688,0.000417,"(-2230.89304636363, -2974.8530169756)"
997,0.000518,0.000495,0.000689,0.000421,"(-1717.8992357807913, -3166.5964849111588)"
998,0.000516,0.000491,0.000689,0.000420,"(-1808.6033754900832, -2779.13038426888)"


In [108]:
X_all = np.array([c1,c2,c3,c4])
X_all
X_all.shape

(4, 1000)

In [109]:
Y_all = centroid
Y_all

array([[-3231.3293467 , -1127.87771457],
       [-3198.54107875,  -744.5100656 ],
       [-3192.9776274 ,  -248.26678827],
       ...,
       [-1717.89923578, -3166.59648491],
       [-1808.60337549, -2779.13038427],
       [   44.43741322,   106.48353609]])

In [110]:
def min_max_scale(x, min, max):
    return (x - min) / (max - min)


def min_max_unscale(x, min, max):
    return x * (max - min) + min

In [111]:
Y_all[:,0] = min_max_scale(Y_all[:,0],-4000,500)
Y_all[:,1] = min_max_scale(Y_all[:,1],-4000,2000)
Y_all

array([[0.1708157 , 0.47868705],
       [0.17810198, 0.54258166],
       [0.17933831, 0.62528887],
       ...,
       [0.5071335 , 0.13890059],
       [0.48697703, 0.20347827],
       [0.89876387, 0.68441392]])

In [112]:
X_all = sklearn.preprocessing.normalize(X_all, axis=0)

In [113]:
X_all

array([[0.47641492, 0.47446433, 0.47588298, ..., 0.47988707, 0.47952697,
        0.573053  ],
       [0.45578912, 0.45516273, 0.4548538 , ..., 0.45878738, 0.45618612,
        0.5528206 ],
       [0.64274013, 0.6441371 , 0.64305085, ..., 0.6379055 , 0.640137  ,
        0.45601097],
       [0.39009005, 0.39089367, 0.3913171 , ..., 0.39025515, 0.39009315,
        0.39755958]], dtype=float32)

In [114]:
X_train = X_all[:,:800]
X_test = X_all[:,800:]

In [115]:
X_train = X_train.reshape(X_train.shape[1], X_train.shape[0])
X_train.shape

(800, 4)

In [116]:
Y_all.shape

(1000, 2)

In [117]:
Y_train = Y_all[:800,:]
Y_test = Y_all[800:,:]

In [118]:
model = LinearRegression()

In [119]:
print(X_train.shape)
print(Y_train.shape)

(800, 4)
(800, 2)


In [120]:
model.fit(X_train,Y_train)

In [121]:
X_test = X_test.reshape(X_test.shape[1], X_test.shape[0])

In [122]:
res = model.predict(X_test)

In [123]:
print(res)

[[0.43545684 0.5218026 ]
 [0.3962842  0.49052507]
 [0.44716713 0.48118874]
 [0.41096064 0.5649424 ]
 [0.42619768 0.5140681 ]
 [0.43020236 0.5199087 ]
 [0.40016726 0.52190787]
 [0.40417597 0.48892808]
 [0.41666356 0.48469085]
 [0.4376428  0.5336965 ]
 [0.42811728 0.51328224]
 [0.4272429  0.51645476]
 [0.40755883 0.5240941 ]
 [0.43512622 0.51389945]
 [0.4421361  0.5218778 ]
 [0.4332233  0.5219051 ]
 [0.43204007 0.5230628 ]
 [0.43223253 0.5216754 ]
 [0.42955464 0.5194081 ]
 [0.40212372 0.5275098 ]
 [0.4251454  0.51865125]
 [0.39816377 0.53330564]
 [0.4289145  0.5260261 ]
 [0.40906313 0.51657337]
 [0.43111995 0.52117026]
 [0.4292045  0.5195257 ]
 [0.43272045 0.52158654]
 [0.43137103 0.5220114 ]
 [0.43467864 0.52360755]
 [0.42976207 0.52107507]
 [0.42569998 0.51289624]
 [0.43221283 0.51810914]
 [0.42197993 0.516677  ]
 [0.42874905 0.4770475 ]
 [0.43261594 0.52673405]
 [0.43218917 0.51831293]
 [0.43256932 0.523325  ]
 [0.43241757 0.52201307]
 [0.43197912 0.5203775 ]
 [0.43181598 0.5204891 ]


In [124]:
Y_test[:,0] = min_max_unscale(Y_test[:,0], -4000,500)
Y_test[:,1] = min_max_unscale(Y_test[:,1], -4000,2000)
Y_test

array([[-3116.67411485,   537.30691584],
       [-3135.52399817,  1334.68572429],
       [-1548.83624842,   985.14330998],
       [-1021.77580516,  1091.48434658],
       [ -523.41865589,   470.51062579],
       [ -184.02556397,   692.2414593 ],
       [ -217.39118154,   502.26075555],
       [ -265.17369808,   267.59358851],
       [ -225.46318102,   -96.57059653],
       [ -106.06701864,     8.45327616],
       [  -83.66058805,   245.32095979],
       [  -48.64463981,   462.62783888],
       [  -21.86068341,   578.44819381],
       [ -213.98215756,   734.78976394],
       [ -250.45877646,   545.34213487],
       [ -186.7152089 ,   413.65275067],
       [ -220.84770337,    35.31971256],
       [-1053.50404879,   832.67315984],
       [-1316.20544531,   895.82379788],
       [-1542.61717589,   951.74490373],
       [-1743.94370435,  1064.70298672],
       [-1999.83744363,  1120.69733841],
       [-2210.99077309,  1219.37857154],
       [-2244.0449803 ,   971.58951204],
       [-2661.31

In [125]:
res[:,0] = min_max_unscale(res[:,0],-4000,500)
res[:,1] = min_max_unscale(res[:,1], -4000,2000)
print(res)

[[-2040.4442   -869.1843 ]
 [-2216.7212  -1056.8496 ]
 [-1987.7479  -1112.8674 ]
 [-2150.6772   -610.34546]
 [-2082.1104   -915.5913 ]
 [-2064.0894   -880.5476 ]
 [-2199.2473   -868.55273]
 [-2181.208   -1066.4316 ]
 [-2125.0142  -1091.855  ]
 [-2030.6073   -797.82104]
 [-2073.4722   -920.30664]
 [-2077.407    -901.2715 ]
 [-2165.9854   -855.4353 ]
 [-2041.932    -916.6033 ]
 [-2010.3875   -868.73315]
 [-2050.495    -868.56934]
 [-2055.8198   -861.62305]
 [-2054.9536   -869.9475 ]
 [-2067.0042   -883.5513 ]
 [-2190.4434   -834.94116]
 [-2086.8457   -888.0925 ]
 [-2208.2632   -800.166  ]
 [-2069.8848   -843.84326]
 [-2159.2158   -900.5598 ]
 [-2059.9602   -872.9785 ]
 [-2068.5798   -882.8457 ]
 [-2052.7578   -870.4807 ]
 [-2058.8303   -867.93164]
 [-2043.946    -858.35474]
 [-2066.0708   -873.54956]
 [-2084.35     -922.62256]
 [-2055.0422   -891.3452 ]
 [-2101.0903   -899.938  ]
 [-2070.6294  -1137.7151 ]
 [-2053.2283   -839.5957 ]
 [-2055.149    -890.1223 ]
 [-2053.438    -860.0498 ]
 

In [126]:
dist = np.linalg.norm(res - Y_test,axis=1)

In [127]:
avg = np.average(dist)
std = np.std(dist)
print(avg,std)

1981.6247285852482 629.4758745132505


In [128]:
print(res[0],Y_test[0])

[-2040.4442  -869.1843] [-3116.67411485   537.30691584]
